In [22]:
import pandas as pd
import requests
import bs4
from time import sleep

In [23]:
months = []
months_range = range(1, 13)
for month_num in months_range:
    req = requests.get(
        "https://www.weathertab.com/en/c/e/{month_num}/united-states/kansas/topeka/")
    month_soup = bs4.BeautifulSoup(req.text)
    day_forecasts = []
    days = page_soup.find_all('td', class_='fct-day')
    for day in days:
        day_forecast = {}
        day_forecast['month'] = f'{month_num}'
        day_forecast['day'] = day.find('div', 'calday').text
        day_forecast['forecast'] = day.find(class_="cal_fct")[
            'class'][1].replace('fct_', '')
        day_forecast['rain'] = day.find(
            class_='cal_fct_text').text.strip()
        months.append(day_forecast)

In [25]:
months

[]

In [24]:
pd.DataFrame(months).sample(10)

ValueError: a must be greater than 0 unless no samples are taken

In [18]:
pd.DataFrame(day_forecasts)

""


In [19]:
for day in months:
    print(f"getting month {day['month']} , day {day['day']}")
    day_url = f"https://www.weathertab.com/en/long-range-weather/e/{day['month']}/{day['day']}/united-states/kansas/topeka/?ref=c"
    day_soup = bs4.BeautifulSoup(requests.get(day_url).text)
    for t in day_soup.find_all('div', class_='F'):
        if (t.text.find('High')):
            day['high_temp_low'] = t.text.strip().split(' to ')[
                0].replace('High Temperature Forecast', '')
            day['high_temp_high'] = t.text.strip().split(
                ' to ')[1].split(' ')[0].replace('\n', '')
        else:
            day['low_temp_low'] = t.text.strip().split(' to ')[
                0].replace('Low Temperature Forecast', '')
            day['low_temp_high'] = t.text.strip().split(
                ' to ')[1].split(' ')[0].replace('\n', '')
    # sleep(1)

In [ ]:
months

In [ ]:
pd.DataFrame(months).to_csv("2024_forecast.csv")

In [26]:
pd.read_csv("2024_forecast.csv")

FileNotFoundError: [Errno 2] No such file or directory: '2024_forecast.csv'